In [ ]:
!pip install caer canaro


In [ ]:
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc

In [ ]:
IMG_SIZE = (80,80)
channels = 1
char_path = r'/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))

#sort in descending order
char_dict = caer.sort_dict(char_dict, descending = True)
char_dict

In [ ]:
character = []
count = 0
for i in char_dict:
    character.append(i[0])
    count +=1
    if count >=10:
        break

character

In [ ]:
#create training data
train = caer.preprocess_from_dir(char_path, character, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)

In [ ]:
len(train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap='gray')
plt.show()

In [ ]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical
#normalize the featureSet ==> 0,1
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(character))

In [ ]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=0.2)

In [ ]:
del train
del featureSet
del labels
gc.collect()

In [ ]:
BATCH_SIZE = 32
EPOCHS=25

In [ ]:
#image generator
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 1)), # Change for custom dataset size
    layers.BatchNormalization(),  # Normalize activations

    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),  # Normalize activations

    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),  # Normalize activations

    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.5),

    
    layers.GlobalAveragePooling2D(),  # Reduces feature maps to a fixed 1D vector
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(10, activation='softmax')  # Change number of classes accordingly
])

model.summary()


In [ ]:
# x_batch, y_batch = next(train_gen)
# print("Batch labels shape:", y_batch.shape)  # Must be (32, 10)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
import tensorflow as tf
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True),
    loss='categorical_crossentropy',  # Use 'binary_crossentropy' for binary classification
    metrics=['accuracy']
)


training = model.fit(train_gen,
                    steps_per_epoch = len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks=callbacks_list
                    )

In [ ]:
character


In [ ]:

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

test_path = '/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/homer_simpson_20.jpg'

# Image size must match training input size
IMG_SIZE = (80, 80)  

def prepare(img_path):
    img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)  # Load image as grayscale
    img = cv.resize(img, IMG_SIZE)  # Resize to match model input
    img = np.reshape(img, (1, 80, 80, 1))  # Add batch dimension
    img = img.astype('float32') / 255.0  # Normalize to [0, 1]
    return img

# Show the image
img = cv.imread(test_path, cv.IMREAD_GRAYSCALE)  # Read in grayscale
plt.imshow(img, cmap='gray')  # Show correctly
plt.axis('off')
plt.show()

# Predict
prepared_img = prepare(test_path)
predictions = model.predict(prepared_img)

# Get class label
predicted_class = np.argmax(predictions)  # Find the highest probability class
print(f"Predicted class: {predicted_class}")

final_char = character[predicted_class]
print(final_char)

In [ ]:
final_char = character[predicted_class]
print(final_char)